In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
df = pd.read_csv("../results/surechembl_smiles_canon_chiral_randomized_patents_l10p_summarizations/surechembl_summarizations_top-100_gpt-3.5-turbo_desc-3500_1.csv")
# df = pd.read_csv("../results/surechembl_smiles_canon_chiral_randomized_patents_l10p_summarizations/test_set/surechembl_summarizations_top-200_gpt-3.5-turbo_desc-3500_1.csv")
# df2 = pd.read_csv("../data/surechembl_smiles_canon_chiral_randomized_patents_l10p_noNA.csv")

In [3]:
df["summarization_sources"] = df["summarization_sources"].apply(literal_eval)

In [4]:
# df['summarization_sources'] contains a dictionary of sources and labels
# convert {k:v, k:v} to [[k,v], [k,v]] so that it can be converted to a dataframe
df_ = df.copy()
df_['summarization_sources'] = df_['summarization_sources'].apply(lambda x: [[k,v] for k,v in x.items()])

In [5]:
# explode summarization_sources
df_ = df_.explode('summarization_sources')

In [6]:
df_['source'] = df_['summarization_sources'].apply(lambda x: x[0])
df_['label'] = df_['summarization_sources'].apply(lambda x: x[1])

In [7]:
def summarizations_to_str(summarizations):
    """
    Converts the GPT summarization to a string of a set that removed duplicates.

    Parameters:
    -----------
    summarizations : dict
        Dictionary of patent IDs to patent summarizations. Format is {patent_id: summarization}.

    Returns:
    --------
    str
        String of a set that removed duplicates.
    
    """
    # split on ' / '
    s_list = summarizations.split(' / ')
    # remove empty strings and 'NA', and "API REQUEST ERROR"
    s_list = [s for s in s_list if s not in ['', 'NA', 'API REQUEST ERROR']]
    return str(set(s_list))

In [8]:
df_["label"] = df_["label"].apply(summarizations_to_str)


In [9]:
df_["label"] = df_["label"].apply(literal_eval)

In [10]:
df_ = df_.explode('label')

In [11]:
df_[["cid", "source", "label"]].to_csv("rand_100_cid_source_label_valid.csv", index=False)